Merge K2 summary statistics catalog with other data from `kplr`
gully

## Query each of the 1658 sources with `kplr` to get meta data, *e.g.* Kepler magnitude.

It adds 16 new columns of information, and one repeated column of information (`input_id`).


### <i class="fa fa-arrow-right fa-1x"></i>   Input:

<i class="fa fa-table fa-2x"></i> 1659 <i class="fa fa-times fa-1x"></i> 12  
<i class="fa fa-file-text fa-2x"></i>`../analysis/K2C02_control_agg_data.csv` (or equivalent).

### Output <i class="fa fa-arrow-right fa-1x"></i>   :

<i class="fa fa-table fa-2x"></i> 1659 <i class="fa fa-times fa-1x"></i> 29  
<i class="fa fa-file-text fa-2x"></i>
`../analysis/K2C02_YSO_kplr_match.csv`

In [5]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina' 
from astropy.utils.console import ProgressBar

### Read in the summary statistics from the previous notebook

In [43]:
ad_clean = pd.read_csv('../analysis/K2C02_YSO_cln_data.csv')
ad_dirty = pd.read_csv('../analysis/K2C02_YSO_simbad_match.csv')

### We want to use `kplr` to fetch auxiliary information about each source.

We don't need to query with kplr for the cleaned data.  Just merge the clean DataFrame with the extra columns from the dirty DataFrame.

In [44]:
ad_dirty.columns

Index(['EPIC_ID', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max',
       'logstd', 'logiqr', 'fname', 'input_id', 'id', 'k2_ra', 'k2_dec',
       'twomass', 'ucac', 'tyc', 'hip', 'kp', 'bmag', 'vmag', 'rmag', 'imag',
       'jmag', 'hmag', 'kmag', 'kepflag', 'sim_2MASS', 'MAIN_ID', 'RA', 'DEC',
       'RA_PREC', 'DEC_PREC', 'COO_ERR_MAJA', 'COO_ERR_MINA', 'COO_ERR_ANGLE',
       'COO_QUAL', 'COO_WAVELENGTH', 'COO_BIBCODE', 'OTYPE', 'ID', 'SP_TYPE',
       'SP_QUAL', 'SP_BIBCODE', 'FLUX_V', 'FLUX_I', 'FLUX_H', 'FLUX_K',
       'INPUT'],
      dtype='object')

In [45]:
extra_cols = list(set(ad_dirty.columns).difference(ad_clean.columns))

Clean the weird byte string B.S.

In [69]:
ad_dirty.SP_TYPE = ad_dirty.SP_TYPE.str[2:-1]

In [70]:
ad_dirty.SP_TYPE.unique()

array([nan, 'F3V', 'G8IV(e)', 'K1III:', '', 'G8IV', 'A0', 'A0V', 'K0IV(e)',
       'G0V', 'K1V(e)', 'A6IV', 'M6', 'F5V', 'A3V', 'G2IV', 'M5.5', 'A1V',
       'K7Ve', 'K5e', 'F6V', 'K0/K1III:', 'F5', 'G9V(e)', 'M2', 'B9V',
       'K7', 'M0', 'F2V', 'F9V', 'A9V+...', 'B8II', 'G1', 'M1', 'A5IV',
       'G8V', 'M5', 'A2m...', 'M3.5', 'M5-6', 'K5V(e)', 'B5III', 'K5',
       'M4.5', 'B8V', 'A2V', 'M4.75', 'M3', 'K3', 'K0', 'F4V', 'M3.75',
       'Me', 'G5', 'M4', 'G4', 'K6', 'K2', 'K1', 'K3/M0e', 'B5V', 'M4.25',
       'A5IV/V', 'M6.5', 'K8', 'K4/M2.5e', 'K4', 'F7V', 'M2.5', 'M6.0',
       'M5.25', 'M8', 'K3:', 'K9', 'M4III', 'M8.5', 'M7.5', 'G9', 'M5III',
       'M7', 'B7Vv...', 'M2III', 'M2e', 'B2III/IV', 'M0.5', 'G9V', 'M6.25',
       'A7', 'B6IV', 'M3.25', 'G7', 'B4', 'K0:', 'G0', 'A1III/IV', 'K0e',
       'K0:Ve', 'B9IIv...', 'G3', 'F2', 'M4.25IIIC', 'G3.5', 'M4.5III',
       'M0:', 'F3.5', 'M3.5III', 'F3', 'F0V', 'M5.0', 'M1.5', 'G3V',
       'M5.75', 'G9IV', 'M7.00', 'M9.00', 'B8/B9II

In [71]:
ad = pd.merge(ad_clean, ad_dirty[['EPIC_ID'] + extra_cols], how='outer', on='EPIC_ID')

In [72]:
ad.OTYPE.unique()

array([nan, 'Star', 'RotV*', 'Candidate_YSO', 'X', 'TTau*', 'pMS*',
       '*inAssoc', 'brownD*', '**', 'RGB*', 'PulsV*delSct', '*inCl',
       'Orion_V*', 'Ae*', 'Em*', 'IR', 'Flare*', 'YSO', 'AGB*',
       'Candidate_brownD*', 'Candidate_TTau*', 'RotV*alf2CVn', 'BYDra',
       'V*', 'RSCVn', 'EB*Algol', 'Cepheid', 'SB*'], dtype=object)

In [73]:
ad = ad[ad_dirty.columns]

In [74]:
k2c2_yso_all_info = ad

In [75]:
k2c2_yso_all_info.tail()

,EPIC_ID,count,mean,std,min,25%,50%,75%,max,logstd,...,OTYPE,ID,SP_TYPE,SP_QUAL,SP_BIBCODE,FLUX_V,FLUX_I,FLUX_H,FLUX_K,INPUT
1673,210282521,1727,1.000547,0.012406,0.969334,0.991273,0.998252,1.008991,1.047247,-1.906380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1674,210282525,1717,1.001416,0.009804,0.986223,0.994732,1.001227,1.007479,1.194411,-2.008590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1675,210282526,1683,1.011602,0.037834,0.901353,0.985588,1.005886,1.035378,1.123121,-1.422115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1676,210282527,1688,1.000744,0.001861,0.997008,0.999152,1.000094,1.002595,1.008156,-2.730217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1677,210282528,1697,0.998154,0.055148,0.859039,0.961421,0.998843,1.024312,1.237276,-1.258470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
k2c2_yso_all_info.to_csv('../analysis/K2C02_YSO_CLN_match.csv', index=False)

# The end.